In [11]:
import sys
sys.path.append('../utils/')

from utils import data_preparation
from tensorflow.python.client import device_lib

In [12]:
# Check CUDA
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4686571803838227775
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2249614951
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2264749562664188980
physical_device_desc: "device: 0, name: Quadro T2000 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]
